I very briefly had the #1 spot on the leaderboard with a score of nearly 80%, because I'm a big, stinky cheater. Leading up to this minor accomplishment, I felt that I had "hacked" this problem, and the effort I put into making things fast and efficient seemed data science-y enough. However, I immediately became racked with guilt and contacted the admins to have my submission removed. In the interest of open knowledge and for the sake of curiosity, however, here is how I destroyed this problem in 20 lines of code and in under a minute.

First of all, you can go download the entire OEIS here: http://oeis.org/wiki/JSON_Format,_Compressed_Files
There's a little bit of parsing to be done, then we very simply create a tree with the classic defaultdict-of-defaultdicts structure. Create the tree by iterating through each sequence, extending the node associated with a number by the next number. For example, consider the Fibonacci sequence 1, 1, 2, 3, 5, 8, ... The root node will have a key '1', the value for which is another node that has a key '1', the value of which is a node that has a key '2', and so on. If there was another sequence, say, 1, 1, 3, ... then that second node mentioned would also have a key '3'.

Once you have this tree built from the data, all ~275,000 of them, go through the test set, split the sequence strings, and for each one, traverse through the tree. When you get to the end of a test sequence, the next number in the sequence is whatever key the current node has. Hopefully there's only one, otherwise there is ambiguity. Not all test sequences are in the OEIS, it seems, and this method only matches the beginning of sequences, thus the ~20% inaccuracy.

I hope people find this interesting or at least amusing!

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import defaultdict

def get_sequences(filename='oeis'):
    with open(filename, 'r') as f:
        for l in f:
            yield l.rstrip(',\n').split(',')

sequences = {s[0]: s[1:] for s in get_sequences()}

def tree():
    return defaultdict(tree)

# This uses each entry in a sequence as keys to traverse the tree
def traverse(s, t):
    if not s:
        return t
    return traverse(s[1:], t[s[0]])

sequence_tree = tree()

# Because the tree is made of defaultdicts, traversing the tree builds it
for name, seq in sequences.iteritems():
    node = traverse(seq, sequence_tree)
    node['name'] = name # Might as well

test_data = pd.read_csv('test.csv')


results = [(traverse(row.Sequence.split(','), sequence_tree).keys() + [0])[0] for _, row in test_data.iterrows()]

submission = pd.DataFrame(results, test_data.Id, ['Last'])
submission.index.name = 'Id'